In [ ]:
%pip install numpy pandas openpyxl matplotlib seaborn plotly folium

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

Codes of Departaments (Administrative divisions of Colombia):

https://geoportal.dane.gov.co/descargas/divipola/DIVIPOLA_Departamentos.xlsx

In [ ]:
divipola = pd.read_excel('DIVIPOLA_Departamentos.xlsx', skiprows=8,header=1).dropna()
divipola['Código'] = divipola['Código'].astype('uint8')
divipola.set_index('Código',inplace=True)
dpto_names = divipola['Nombre'].to_dict()
dpto_lat = divipola['LATITUD'].to_dict()
dpto_lon = divipola['LONGITUD'].to_dict()

Population per Department:
https://en.wikipedia.org/wiki/List_of_Colombian_departments_by_population

In [ ]:
poblacion_dpto = pd.read_csv('población.csv')
poblacion_dpto['Department'] = poblacion_dpto['Department'].str.upper()
poblacion_dpto['CODE'] = poblacion_dpto['Department'].map({v:k for k,v in dpto_names.items()})
poblacion_dpto.loc[0,'CODE'] = 11
poblacion_dpto.loc[28,'CODE'] = 88
poblacion_dpto['CODE'] = poblacion_dpto['CODE'].astype('uint8')
poblacion_dpto.set_index('CODE',inplace=True)

Encuesta Anual Manufacturera (Annual Industry Survey):

2018: https://microdatos.dane.gov.co/index.php/catalog/650/download/12432

2019: https://microdatos.dane.gov.co/index.php/catalog/694/download/14186

2020: https://microdatos.dane.gov.co/index.php/catalog/724/download/20937

2021: https://microdatos.dane.gov.co/index.php/catalog/802/download/22877

In [ ]:

def get_industry_employees(file_path: str, delimiter: str):
    eam = pd.read_csv(file_path, delimiter=delimiter)
    eam = eam[["nordemp","nordest", "ciiu4", "dpto", "c4r4c9t","c4r4c10t"]]
    industry_employees = eam.groupby("dpto").aggregate({'c4r4c9t': 'sum','c4r4c10t': 'sum'})
    return industry_employees

file_format = "EAM_{0}.csv"
years = [2018,2019,2020,2021]
delimiters = [';',';',',',',']
industry_employees = {}
for year, delimiter in zip(years, delimiters):
    industry_employees[year] = get_industry_employees(file_format.format(year), delimiter)

In [ ]:
for year in years:
    industry_employees[2021][f"Mujeres {year}"] = industry_employees[year]["c4r4c9t"]
    industry_employees[2021][f"Hombres {year}"] = industry_employees[year]["c4r4c10t"]
industry_employees = industry_employees[2021].drop(["c4r4c9t","c4r4c10t"],axis=1)

In [ ]:
industry_employees["DEPARTAMENTO"] = industry_employees.index.map(dpto_names)
industry_employees["LATITUD"] = industry_employees.index.map(dpto_lat)
industry_employees["LONGITUD"] = industry_employees.index.map(dpto_lon)
industry_employees["Población"] = poblacion_dpto["Population (2020)"]
industry_employees["Porcentaje"] = (industry_employees['Hombres 2021']+industry_employees['Mujeres 2021']) / industry_employees["Población"]

In [ ]:
from plotly import express as px
format_col = '{} {}'

def empleados_dpto(cod: int):
    hombres_dpto = industry_employees.loc[cod,[format_col.format("Hombres",year) for year in years]]
    hombres_dpto.rename({format_col.format("Hombres",year): year for year in years}, inplace=True)
    mujeres_dpto = industry_employees.loc[cod,[format_col.format("Mujeres",year) for year in years]]
    mujeres_dpto.rename({format_col.format("Mujeres",year): year for year in years}, inplace=True)
    return pd.DataFrame([hombres_dpto.rename('Men'),mujeres_dpto.rename('Women')]).T

def dpto_popup(cod: int):
    df = empleados_dpto(cod)
    title = f"{industry_employees.loc[cod,'DEPARTAMENTO']}: Employees in Manufacturing"
    fig = px.bar(df, color_discrete_sequence=['blue','magenta'],
                 labels={'index': 'Year','value':'Employees','variable': 'Gender'},
                 barmode='group', title=title)
    return fig

for cod in industry_employees.index:
    fig = dpto_popup(cod)
    fig.update_layout(margin=dict(t=30,l=10,b=10,r=10))
    fig.write_html(f"bar_charts/{cod}.html", include_plotlyjs='cdn') # include_plotly makes map.html much lighter and loast faster.


In [ ]:
min_employed = industry_employees['Porcentaje'].min()
max_employed = industry_employees['Porcentaje'].max()

Colombia GeoJson: https://gist.github.com/john-guerra/43c7656821069d00dcbc

In [ ]:
import folium
from branca.colormap import linear
from branca.element import IFrame

colormap = linear.YlGn_09.scale(
    0, max_employed * 100
)

def make_style(feature: dict):
    cod = int(feature["properties"]["DPTO"])
    if cod in industry_employees.index:
        c = colormap(industry_employees.loc[cod,'Porcentaje'] * 100)
    else:
        c= colormap(0)
    return {"fillColor": c, "weight":1, "fillOpacity": 0.9}


loc = 'Population in Colombia Employed by Manufacturing Industry'
title_html = f'''
             <h3 align="center" style="font-size:16px"><b>{loc}</b></h3>
             ''' 

m = folium.Map(location=(4.570868, -74.297333),zoom_start=5, tiles="cartodb positron")
folium.GeoJson('Colombia.geo.json', style_function=make_style).add_to(m)
colormap.caption ="Percentage of the Population Employed (2021)"
colormap.add_to(m)
for cod in industry_employees.index:
    filepath = f"bar_charts/{cod}.html"
    with open(filepath , encoding='utf-8') as f:
        html = f.read()
    iframe = IFrame(html=html, width=500, height=300)
    popup = folium.Popup(iframe, max_width=500)
    folium.Marker([industry_employees.loc[cod,"LATITUD"],industry_employees.loc[cod,"LONGITUD"]],
                  popup=popup).add_to(m)


m.get_root().html.add_child(folium.Element(title_html))
m.save('map.html')